Статистический анализ данных сервиса аренды самокатов GoFast

Необходимо проанализировать данные и проверить некоторые гипотезы, которые могут помочь бизнесу вырасти.

Даны 3 таблицы:

Пользователи — users_go.csv

user_id	уникальный идентификатор пользователя
name	имя пользователя
age	возраст
city	город
subscription_type	тип подписки (free, ultra)

Поездки — rides_go.csv

user_id	уникальный идентификатор пользователя
distance	расстояние, которое пользователь проехал в текущей сессии (в метрах)
duration	продолжительность сессии (в минутах) — время с того момента, как пользователь нажал кнопку «Начать поездку» до момента, как он нажал кнопку «Завершить поездку»
date	дата совершения поездки

Подписки — subscriptions_go.csv

subscription_type	тип подписки
minute_price	стоимость одной минуты поездки по данной подписке
start_ride_price	стоимость начала поездки
subscription_fee	стоимость ежемесячного платежа

Необходимо:
1. Провести предобработку данных, включая поиск пропущенных значений и дубликатов


2. Провести исследовательский анализ данных с описанием и визуализацией общей информации о пользователях и поездках


3. Объединить таблицы в одну, а также создать две таблицы для пользователей с платной подпиской и без нее


4. Визуализировать информацию о расстоянии и времени поездок для пользователей обеих категорий


5. Подсчитать выручку для каждого пользователя за каждый месяц


6. Проверить 4 гипотезы:    
    1. тратят ли пользователи с подпиской больше времени на поездки?;
    2. среднее расстояние, которое проезжают пользователи с подпиской за одну поездку, не превышает 3130 метров?;    
    3. помесячная выручка от пользователей с подпиской по месяцам выше, чем выручка от пользователей без подписки?;    
    4. количество обращений в техподдержку снизилось после обновления серверов
    
    
7. Выяснить, какое минимальное количество промокодов надо разослать, чтобы минимум 100 клиентов продолжили пользоваться подпиской


8. Оценить вероятность того, что после рассылки 1 млн. уведомлений, 399.5 тыс. пользователей откроют уведомление

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats as st
from scipy.stats import binom
from math import sqrt
import matplotlib.pyplot as plt

#### Шаг 1. Загрузка данных

In [ ]:
users = pd.read_csv('/datasets/users_go.csv')
rides = pd.read_csv('/datasets/rides_go.csv')
subscr = pd.read_csv('/datasets/subscriptions_go.csv')

In [ ]:
display(users)
display(rides)
display(subscr)

#### Шаг 2. Предобработка данных

Приведение столбца `date` к типу даты *pandas*:

In [ ]:
rides['date'] = pd.to_datetime(rides['date'])

Добавление столбца с номером месяца:

In [ ]:
rides['month'] = rides['date'].dt.month

Проверим пропуски:

In [ ]:
for column in users.columns:
    missing_data = users[column].isna().sum()
    print(f'Пропусков в столбце "{column}":{missing_data}')

In [ ]:
for column in rides.columns:
    missing_data = rides[column].isna().sum()
    print(f'Пропусков в столбце "{column}":{missing_data}')

In [ ]:
for column in subscr.columns:
    missing_data = subscr[column].isna().sum()
    print(f'Пропусков в столбце "{column}":{missing_data}')

Проверим дубликаты:

In [ ]:
users.duplicated().sum()

In [ ]:
rides.duplicated().sum()

In [ ]:
subscr.duplicated().sum()

Удалим дубликаты:

In [ ]:
users.drop_duplicates(inplace = True)

#### Шаг 3. Исследовательский анализ данных

Построим гистограммы по всем данным:

In [ ]:
users['age'].hist(figsize=(4,4));

Чаще всего услугами сервиса пользуются люди в возрасте около 25 лет.

In [ ]:
rides[['distance','duration','month']].hist(figsize=(9,9));

Чаще всего проезжают около 3000 м., длительность поездки составляет 20 мин.
По какой-то загадочной причине в декабре и январе происходит резкий рост использования сервиса.

Проверим частоту встречаемости городов:

In [ ]:
users['city'].value_counts()

In [ ]:
users['city'].value_counts().plot(kind='bar', title='Частота встречаемости городов', 
                                  ylabel='Количество пользователей', xlabel='Город')

Посчитаем количество пользователей с подпиской и без:

In [ ]:
users['subscription_type'].value_counts()

Визуализируем эти данные:

In [ ]:
labels = ['free', 'ultra']
values = [users['subscription_type'].value_counts()[0], users['subscription_type'].value_counts()[1]]  

plt.pie(values, labels=labels,startangle=90,autopct='%1.2f%%');

Посчитаем средние некоторый показателей:

In [ ]:
users['age'].mean()

In [ ]:
rides['distance'].mean()

In [ ]:
rides['duration'].mean()

Вывод: Средний возраст пользователя 25 лет, средняя длина поездки 3070 м., средняя длительность поездки 18 мин. Самый частый город - Пятигорск, наименее встречающийся город - Москва. В течение года количество поездок по месяцам примерно одинаковое за исключением декабря и января, когда наблюдается резкий рост.

#### Шаг 4. Объединение данных

Создадим общую таблицу:

In [ ]:
df = users.merge(rides, on='user_id').merge(subscr, on='subscription_type')
df

Создадим раздельные таблицы для пользователей с подпиской и без:

In [ ]:
subscr_free = df.query('subscription_type == "free"')

In [ ]:
subscr_ultra = df.query('subscription_type == "ultra"')

Выведем гистограммы для пройденного расстояния для пользователей без подписки и с подпиской:

In [ ]:
subscr_free['distance'].hist();

In [ ]:
subscr_ultra['distance'].hist();

In [ ]:
print(subscr_free['distance'].median())
print(subscr_ultra['distance'].median())

Медианы расстояний почти не различаются, однако по гистограммам видно, что пользователи с подпиской почти не проезжают меньше 2000 м.

Выведем гистограммы для времени поездки для пользователей без подписки и с подпиской:

In [ ]:
subscr_free['duration'].hist();

In [ ]:
subscr_ultra['duration'].hist();

In [ ]:
print(subscr_free['duration'].mean())
print(subscr_ultra['duration'].mean())

Выведем объединенную гистограмму дальности поездки для пользователей *ultra* и *free*

In [ ]:
plt.figure(figsize=(9,6))
plt.hist(subscr_free['distance'], label='free', bins=20, alpha=0.8)
plt.hist(subscr_ultra['distance'], label='ultra',bins=20, alpha=0.8)
plt.legend()
plt.show()

Пользователи без подписки ездят больше. Наблюдается небольшой хвост от 0 до 1500 м для обеих групп. Это можно объяснить, вероятно, близким расположением парковок в популярном месте, либо наличием этих парковок в удобных местах для небольших поездкок, например, в магазин. 

Выведем объединенную гистограмму длительности поездки для пользователей *ultra* и *free*

In [ ]:
plt.figure(figsize=(9,6))
plt.hist(subscr_free['duration'], label='free', bins=20, alpha=0.8)
plt.hist(subscr_ultra['duration'], label='ultra',bins=20, alpha=0.8)
plt.legend()
plt.show()

Длительность поездок для обеих групп выглядит нормально. Бесплатные пользователи почти всегда ездят дольше, за исключением очень длительных поездок более 35 мин.

#### Шаг 5. Подсчёт выручки

Найдем суммарное расстояние, количество поездок и суммарное время для каждого пользователя за каждый месяц:

In [ ]:
sum_per_month = df.groupby(['user_id','month']).agg(total_distance=('distance', 'sum'),
    total_duration=('duration', 'sum'),
    ride_count=('distance', 'count')
).reset_index().merge(users[['user_id', 'subscription_type']], on='user_id')

sum_per_month

Найдем ежемесячную выручку:

In [ ]:
#Добавим данные о стоимости
sum_per_month = sum_per_month.merge(subscr, on='subscription_type')

#Формула расчета выручки с пользователя
sum_per_month['income'] = (sum_per_month['ride_count']*sum_per_month['start_ride_price']
+sum_per_month['total_duration']*sum_per_month['minute_price']+sum_per_month['start_ride_price']
+sum_per_month['subscription_fee']
                          )
#Удалим лишнее
sum_per_month.drop(['minute_price', 'start_ride_price', 'subscription_fee'], axis = 1, inplace=True)

#Округлим вверх и приведем к типу int
sum_per_month['income'] = np.ceil(sum_per_month['income']).astype(int)

sum_per_month

#### Шаг 6. Проверка гипотез

6.1

H0: пользователи с подпиской тратят больше времени на поездки.

H1: пользователи с подпиской тратят меньше или столько же времени на поездки.

In [ ]:
alpha = 0.05

#Одностороннимй
result = st.ttest_ind(sum_per_month.query('subscription_type=="ultra"')['total_duration'],
                      sum_per_month.query('subscription_type=="free"')['total_duration'], alternative='less' )

print('p-значение',result.pvalue)

print('Длительность поездки с подпиской:',np.mean(sum_per_month.query('subscription_type=="ultra"')['total_duration']))    
print('Длительность поездки без подписки:',np.mean(sum_per_month.query('subscription_type=="free"')['total_duration']))

if result.pvalue < alpha:
    print("Отвергаем нулевую гипотезу: пользователи с подпиской тратят меньше или столько же времени на поездки.")
else:
    print("Не отвергаем нулевую гипотезу.")

    




6.2

H0: среднее расстояние, которое проезжают пользователи с подпиской за одну поездку, не превышает 3130 метров.

H1: среднее расстояние, которое проезжают пользователи с подпиской за одну поездку, превышает 3130 метров.

In [ ]:
value=3130

result=st.ttest_1samp(sum_per_month.query('subscription_type=="ultra"')['total_distance'],value, alternative='greater')

print('Пройденная дистанция с подпиской:',np.mean(sum_per_month.query('subscription_type=="ultra"')['total_distance']))

print('p-значение',result.pvalue)


if result.pvalue < alpha:
    print("Отвергаем нулевую гипотезу: среднее расстояние превышает 3130 метров.")
else:
    print("Не отвергаем нулевую гипотезу")

Вывод: Пользователи с подпиской повышают амортизацию бизнеса.

6.3.

H0: Помесячная выручка от пользователей с подпиской по месяцам выше, чем выручка от пользователей без подписки.

H1: Помесячная выручка от пользователей с подпиской по месяцам не выше, чем выручка от пользователей без подписки.

In [ ]:
result = st.ttest_ind(sum_per_month.query('subscription_type=="ultra"')['income'],
                      sum_per_month.query('subscription_type=="free"')['income'], alternative='less' )

print('p-значение',result.pvalue)

print('Выручка с подпиской', np.mean(sum_per_month.query('subscription_type=="ultra"')['income']))
print('Выручка без подписки', np.mean(sum_per_month.query('subscription_type=="free"')['income']))

if result.pvalue < alpha:
    print("Отвергаем нулевую гипотезу: пользователи без подписки приносят больше выручки.")
else:
    print("Не отвергаем нулевую гипотезу.")

Вывод: хоть пользователи без подписки и приносят больше прибыли, пользователи с подпиской приносят стабильную прибыль.

6.4

Двухвыборочный t-тест для зависимых выборок


H0: Количество обращений в техподдержку снизилось.

H1: Обращений в техподдержку столько же или больше. #Как правильнее здесь написать: "не снизилось" или "столько же или больше"?

In [ ]:
# st.ttest_rel(new_support, old_support, alternative='greater')

#### Шаг 7. Распределения

Отделу маркетинга GoFast поставили задачу: нужно провести акцию с раздачей промокодов на один бесплатный месяц подписки, в рамках которой как минимум 100 существующих клиентов должны продлить эту подписку. То есть по завершении периода действия подписки пользователь может либо отказаться от неё, либо продлить, совершив соответствующий платёж. Эта акция уже проводилась ранее и по итогу выяснилось, что после бесплатного пробного периода подписку продлевают 10 % пользователей. Выясните, какое минимальное количество промокодов нужно разослать, чтобы вероятность не выполнить план была примерно 5 %. Подберите параметры распределения, описывающего эту ситуацию, постройте график распределения и сформулируйте ответ на вопрос о количестве промокодов.

In [ ]:
p = 0.1 # Вероятность успеха
success = 0.05 # Вероятность неуспеха
clients = 100 # Минимальное количество успехов

n = clients
# С помощью цилка будем подбирать n, пока вероятность не достигнет требуемой
while True:
    result = st.binom.cdf(clients-1,n,p)
    if result <= success:
        break
    n += 1
    
print(n)

x = range(0, n)
y = binom.pmf(x, n, p)

plt.figure(figsize=(12, 9))
plt.bar(x, y)
plt.grid(axis='y')
plt.show()

По расчетам необходимо разослать 1161 промокод, но по графику это как-то неочевидно. 

Вообще дальше у меня тоже как-то непонятно получилось

7.2 Отдел маркетинга рассылает клиентам push-уведомления в мобильном приложении. Клиенты могут открыть его или не открывать. Известно, что уведомления открывают около 40 % получивших клиентов. Отдел планирует разослать 1 млн уведомлений. С помощью аппроксимации постройте примерный график распределения и оцените вероятность того, что уведомление откроют не более 399,5 тыс. пользователей.

In [ ]:
n = 1000000   # тысяч уведомлений 
p = 0.4 # вероятность успеха

mu = n*p  

sigma = (n*p*(1-p))**0.5 

distr = st.norm(mu,sigma)

result = distr.cdf(399500)

print('Точное значение вероятности:', result)

In [ ]:
def linspace(start, stop, num):
    step = (stop - start) / (num - 1)
    result = []
    for i in range(num):
        result.append(start + step * i)
    return result

left = int(n * p - 4 * sqrt(n * p * (1 - p)))
right = int(n * p + 4 * sqrt(n * p * (1 - p)))

x = list(range(max(0, left), right))

plt.figure(figsize=(12, 9))
# рисуем график биномиального распределения
ax = plt.subplot()
# используем синие о-образные маркеры и размер маркера 5
ax.plot(x, binom.pmf(x, n, p), '-o', ms=5,
	      label='вероятности биномиального распределения')
 
# рисуем график нормального распределения:
# получим список из 100 чисел от left до right
x = linspace(left, right, 100)
# используем красную линию ширины 3 и прозрачности 0.3
ax.plot(x, st.norm.pdf(x, mu, sigma), 'r-', lw=3, alpha=0.3, label='плотность вероятности нормального распределения')
 
# зададим подписи по оси x в пределах ± 3 ст.отклонений от мат.ожидания
# (последний аргумент в методе range - интервал между подписями)
x_ticks = list(range(int(mu - 3 * sigma), int(mu + 3 * sigma), 500))
ax.set_xticks(x_ticks)

# настраиваем оптимальное расположение легенды и выводим график

ax.legend(loc='best')
plt.show()


Вывод:

На этапе предобработки обнаружено незначительное (31) количество дубликатов, которые были удалены.

При анализе данных установлено следующее:

1. Средний возраст пользователя 25 лет.
2. Длительность поездки составляет около 18 мин. 
3. В декабре и январе происходит резкий рост использования сервиса. Вероятно, это связано с праздниками и теплыми зимами в некоторых городах.
4. Самый частый город - Пятигорск, наименее встречающийся город - Москва.
5. Доля пользователей с подпиской *free* 54.4%, доля пользователей с подпиской *ultra* 45.6%. 

Подтвердившиеся гипотезы:
1. Пользователи с подпиской тратят меньше или столько же времени на поездки
2. Среднее расстояние превышает 3130 метров
3. Пользователи без подписки приносят больше выручки

Таким образом получается, что пользователи с подпиской тратят меньше денег и при этом приводят к еще большим тратам у компании (у самоката предел 3130 м.).